In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture --no-stderr
!pip install python-dotenv langchain-community langchain-core langchain langchain-openai langchain-chroma

In [ ]:
# 환경변수 설정

In [ ]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [ ]:
# .env 파일에서 환경 변수 로드 (.env 파일에는 OPENAI API 키값을 적으면 됩니다. -> OPENAI_API_KEY=...)
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

In [ ]:
# <이전 대화를 포함한 메시지 전달>

# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")

# 프롬프트 템플릿 정의: 금융 상담 역할
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다."),
        ("placeholder", "{messages}"),  # 대화 이력 추가
    ]
)

# 프롬프트와 모델을 연결하여 체인 생성
chain = prompt | chat

# 이전 대화를 포함한 메시지 전달
ai_msg = chain.invoke(
    {
        "messages": [
            ("human", "저축을 늘리기 위해 무엇을 할 수 있나요?"),  # 사용자의 첫 질문
            ("ai", "저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요."),  # 챗봇의 답변
            ("human", "방금 뭐라고 했나요?"),  # 사용자의 재확인 질문
        ],
    }
)
print(ai_msg.content)  # 챗봇의 응답 출력


저축을 늘리기 위해 먼저 저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하는 방법을 추천했습니다. 이 방법은 저축을 습관화하는 데 도움을 줄 수 있습니다. 추가적인 방법으로는 다음과 같은 것들이 있습니다:

1. **예산 수립**: 수입과 지출을 분석하여 필요한 지출과 불필요한 지출을 구분하고 예산을 세워 보세요.

2. **지출 줄이기**: 불필요한 소비를 줄이고, 필요한 비용을 최적화해 보세요. 예를 들어, 외식 횟수를 줄이거나 할인 상품을 이용하는 방법이 있습니다.

3. **고금리 저축 계좌 이용**: 일반 저축 계좌보다 이율이 높은 고금리 저축 계좌로 자금을 넣어 두면 더 많은 이자를 받을 수 있습니다.

4. **부가 수입원 찾기**: 추가적인 수입원을 만들면 저축을 더 늘릴 수 있습니다. 프리랜스 일이나 취미로 시작하는 소규모 사업이 될 수 있습니다.

5. **재정 목표 설정**: 단기적, 중기적, 장기적 목표를 명확하게 설정하고 이를 달성하기 위한 계획을 세워보세요.

이런 방법들을 통해 저축을 효과적으로 늘릴 수 있습니다.


In [ ]:
# <`ChatMessageHistory`를 사용한 메시지 관리>

from langchain_community.chat_message_histories import ChatMessageHistory

# 대화 이력 저장을 위한 클래스 초기화
chat_history = ChatMessageHistory()

# 사용자 메시지 추가
chat_history.add_user_message("저축을 늘리기 위해 무엇을 할 수 있나요?")
chat_history.add_ai_message("저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.")

# 새로운 질문 추가 후 다시 체인 실행
chat_history.add_user_message("방금 뭐라고 했나요?")
ai_response = chain.invoke({"messages": chat_history.messages})
print(ai_response.content)  # 챗봇은 이전 메시지를 기억하여 답변합니다.

저축을 늘리기 위해 저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하는 것이 좋다고 말씀드렸습니다. 이렇게 하면 규칙적으로 저축할 수 있는 습관을 기를 수 있습니다. 추가적으로, 소비를 줄이는 방법이나 불필요한 지출을 점검하는 것도 저축을 늘리는 데 도움이 됩니다. 더 구체적인 조언이 필요하시다면 말씀해 주세요!


In [ ]:
# <` RunnableWithMessageHistory`를 사용한 메시지 관리>

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# 시스템 메시지와 대화 이력을 사용하는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 모든 질문에 최선을 다해 답변하십시오."),
        ("placeholder", "{chat_history}"),  # 이전 대화 이력
        ("human", "{input}"),  # 사용자의 새로운 질문
    ]
)

# 대화 이력을 관리할 체인 설정
chat_history = ChatMessageHistory()
chain = prompt | chat

# RunnableWithMessageHistory 클래스를 사용해 체인을 감쌉니다
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,  # 세션 ID에 따라 대화 이력을 불러오는 함수
    input_messages_key="input",  # 입력 메시지의 키 설정
    history_messages_key="chat_history",  # 대화 이력의 키 설정
)

# RunnableWithMessageHistory 클래스를 사용해 체인을 감쌉니다
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,  # 세션 ID에 따라 대화 이력을 불러오는 함수
    input_messages_key="input",  # 입력 메시지의 키 설정
    history_messages_key="chat_history",  # 대화 이력의 키 설정
)

# 질문 메시지 체인 실행
chain_with_message_history.invoke(
    {"input": "저축을 늘리기 위해 무엇을 할 수 있나요?"},
    {"configurable": {"session_id": "unused"}},
).content


'저축을 늘리기 위한 몇 가지 방법을 소개합니다:\n\n1. **예산 수립하기**: 월별 예산을 세워 수입과 지출을 명확하게 파악하세요. 고정 지출과 변동 지출을 분리한 후, 불필요한 지출을 줄이는 방법을 찾아보세요.\n\n2. **비상금 마련하기**: 예기치 않은 상황에 대비해 최소 3~6개월 분의 생활비를 비상금으로 저축하세요. 이를 통해 갑작스러운 지출에 대응할 수 있습니다.\n\n3. **정기 저축**: 월급이 들어오는 즉시 일정 금액을 저축 계좌로 자동 이체하면, 저축을 더 쉽게 할 수 있습니다.\n\n4. **소비 습관 점검**: 취미나 외식 등에서의 지출 패턴을 분석하고, 필요한 부분만 소비하도록 합니다. 더 건강한 소비 습관을 기르는 것이 중요합니다.\n\n5. **금융 상품 활용**: 고금리 저축 계좌나 재테크 상품을 통해 더욱 높은 이자를 받을 수 있도록 다양한 금융 상품을 고려해보세요.\n\n6. **부가 소득 창출**: 아르바이트나 프리랜스 일 등을 통해 추가 수입을 마련하여 저축에 활용할 수 있습니다.\n\n7. **목표 설정하기**: 구체적인 저축 목표를 세우고, 그 목표를 달성하기 위한 계획을 수립하세요. 자금 목표를 명확히 하면 저축 의욕이 더 높아질 수 있습니다.\n\n8. **비교 구매하기**: 물건을 구매할 때 여러 판매처를 비교하고 최저가를 찾아보세요. 세일이나 할인 이벤트를 활용하는 것도 좋습니다.\n\n9. **신용카드 사용 조절**: 신용카드 사용을 최소화하고, 필요한 경우 한정된 카드만 사용하는 것이 도움이 됩니다. 카드 사용 시 지출 내역을 꼼꼼히 기록하세요.\n\n이러한 방법들을 통해 저축 습관을 들이고, 재정적으로 더 안정된 미래를 준비할 수 있습니다.'

In [ ]:
# 새로운 입력 메시지를 추가하고 체인을 실행
chain_with_message_history.invoke(
    {"input": "내가 방금 뭐라고 했나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "unused"}}  # 세션 ID 설정
).content


'당신은 "저축을 늘리기 위해 무엇을 할 수 있나요?"라고 질문하셨습니다. 저축을 늘리기 위한 다양한 방법을 제안해드렸습니다. 추가적인 질문이나 더 알고 싶은 내용이 있으시면 말씀해 주세요!'

In [ ]:
# <메시지 트리밍 예제>

# 라이브러리 불러오기
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# 메시지 트리밍 유틸리티 설정
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

# 트리밍된 대화 이력과 함께 체인 실행
chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter("chat_history") | trimmer)
    | prompt
    | chat
)

# 트리밍된 대화 이력을 사용하는 체인 설정
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# 새로운 대화 내용 추가 후 체인 실행
chain_with_trimmed_history.invoke(
    {"input": "저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "finance_session_1"}}  # 세션 ID 설정
)



AIMessage(content='5년 내에 집을 구매하기 위한 재정 계획을 세우는 것은 중요한 단계입니다. 다음은 효과적인 계획을 위한 몇 가지 단계입니다:\n\n1. **목표 설정**:\n   - 구매할 집의 가격을 예상합니다. 지역에 따라 다양한 가격대가 있으므로, 관심 있는 지역의 평균 집값을 조사하세요.\n\n2. **다운 페이먼트 계산**:\n   - 일반적으로 집값의 10-20%를 다운 페이먼트로 요구합니다. 예를 들어, 3억 원짜리 집의 경우, 3천만 원에서 6천만 원 정도가 필요합니다.\n\n3. **예산 작성**:\n   - 현재의 소득과 지출을 파악하여 저축할 수 있는 금액을 계산하세요. 매달 얼마를 저축할 수 있는지 확인하는 것이 중요합니다.\n\n4. **저축 계획**:\n   - 5년 동안 필요한 다운 페이먼트를 목표로 매월 저축해야 할 금액을 계산합니다. 예를 들어, 3천만 원의 다운 페이먼트를 위해 5년(60개월) 동안 저축해야 할 경우, 매월 약 50만 원을 저축해야 합니다.\n\n5. **저축 계좌 선택**:\n   - 일반 저축계좌, 고이율 저축계좌, 또는 적금 상품을 고려하여 이자를 통해 저축액을 늘리세요.\n\n6. **부동산 시장 연구**:\n   - 주택 시장의 변동성을 이해하고, 특정 지역의 부동산 가격 상승률을 고려해 적절한 시기에 구매할 수 있도록 조사하세요.\n\n7. **전문가 상담**:\n   - 부동산 상담사나 재정 상담사와 상담하여 개인의 상황에 맞는 구체적인 전략을 세우는 것이 좋습니다.\n\n8. **신용 점수 관리**:\n   - 대출을 받을 때 유리한 조건을 위해 신용 점수를 관리하세요. 신용 점수를 높이기 위해서는 연체 없이 빚을 갚고, 신용 카드 사용을 적절히 관리하는 것이 중요합니다.\n\n위의 단계를 따라주시고, 필요에 따라 개인의 재정 상황에 맞게 조정하면 좋습니다. 추가 질문이 있으시면 언제든지 말씀해 주세요!', additional_kwargs={'refusal': None}, resp

In [ ]:
# 새로운 입력 메시지를 추가하고 체인을 실행
chain_with_trimmed_history.invoke(
    {"input": "내가 방금 뭐라고 했나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "finance_session_1"}}  # 세션 ID 설정
).content


'당신은 "저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?"라고 말씀하셨습니다. 이 질문에 대해 저는 집 구매를 위한 재정 계획의 단계들을 설명해드렸습니다. 추가로 궁금하신 점이나 더 알고 싶은 내용이 있으시면 말씀해 주세요!'

In [ ]:
# <이전 대화 요약 내용 기반으로 답변하기>

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    # 대화를 요약하기 위한 프롬프트 템플릿 설정
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),  # 이전 대화 이력
            (
                "user",
                "이전 대화를 요약해 주세요. 가능한 한 많은 세부 정보를 포함하십시오.",  # 요약 요청 메시지
            ),
        ]
    )

    # 요약 체인 생성 및 실행
    summarization_chain = summarization_prompt | chat
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})
    chat_history.clear()  # 요약 후 이전 대화 삭제
    chat_history.add_message(summary_message)  # 요약된 메시지를 대화 이력에 추가
    return True

In [ ]:
# 대화 요약을 처리하는 체인 설정
chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

# 요약된 대화를 기반으로 새로운 질문에 응답
print(chain_with_summarization.invoke(
    {"input": "저에게 어떤 재정적 조언을 해주셨나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "unused"}}  # 세션 ID 설정
).content)


제가 제공한 재정적 조언은 다음과 같습니다:

1. **저축을 늘리기 위한 방법**:
   - **예산 수립**: 월별 예산을 만들어 수입과 지출을 분석하고 불필요한 지출을 줄이세요.
   - **비상금 마련**: 생활비의 3~6개월 분량을 비상금으로 확보하여 예기치 않은 상황에 대비하세요.
   - **정기 저축**: 월급이 들어오면 자동 이체를 설정하여 저축하는 습관을 기르세요.
   - **소비 습관 점검**: 소비 패턴을 분석하고 필요한 부분만 지출하도록 하세요.
   - **금융 상품 활용**: 고금리 저축 계좌나 재테크 상품 등으로 더 높은 이자를 받을 수 있도록 하세요.
   - **부가 소득 창출**: 아르바이트나 프리랜서 업무를 통해 추가 소득을 만들어 저축하세요.
   - **목표 설정**: 구체적인 저축 목표를 세우고 이를 달성하기 위한 계획을 수립하세요.
   - **비교 구매**: 구매할 때 여러 판매처를 비교하고 세일이나 할인 이벤트를 활용하세요.
   - **신용카드 사용 조절**: 신용카드를 최소한으로 사용하고 지출 내역을 기록하세요.

2. **5년 내에 집 구매를 위한 재정 계획**:
   - **목표 설정**: 구매할 집의 가격을 예상하고 해당 지역의 평균 집값을 조사하세요.
   - **다운 페이먼트 계산**: 집값의 10-20%를 다운 페이먼트로 준비할 계획을 세우세요.
   - **예산 작성**: 현재 소득과 지출을 파악하여 매월 저축 가능한 금액을 계산하세요.
   - **저축 계획**: 다운 페이먼트를 목표로 정기적으로 저축할 금액을 정하세요.
   - **저축 계좌 선택**: 일반 저축계좌, 고이율 저축계좌 등 다양한 저축 수단을 고려하세요.
   - **부동산 시장 연구**: 주택 시장의 트렌드를 이해하고 적절한 시기에 구매할 수 있도록 하세요.
   - **전문가 상담**: 부동산 전문가나 재정 상담사와 상담하여 개인 상황에 맞는 전략을 세우세요.
   - **신용 점수 관리**: 좋은 대출 조건을 확보하기 위해